In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import os
import pydotplus

## 数据处理

In [48]:
#sl:satisfaction_level---False:MinMaxScaler;True:StandardScaler
#le:last_evaluation---False:MinMaxScaler;True:StandardScaler
#npr:number_project---False:MinMaxScaler;True:StandardScaler
#amh:average_monthly_hours--False:MinMaxScaler;True:StandardScaler
#tsc:time_spend_company--False:MinMaxScaler;True:StandardScaler
#wa:Work_accident--False:MinMaxScaler;True:StandardScaler
#pl5:promotion_last_5years--False:MinMaxScaler;True:StandardScaler
#dp:department--False:LabelEncoding;True:OneHotEncoding
#slr:salary--False:LabelEncoding;True:OneHotEncoding
def map_salary(s):
    d=dict([("low",0),("medium",1),("high",2)])
    return d.get(s,0)
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv("./data/HR.csv")
    
    #1、清洗数据
    df=df.dropna(subset=["satisfaction_level","last_evaluation"])
    df=df[df["satisfaction_level"]<=1][df["salary"]!="nme"]
    #2、得到标注
    label = df["left"]
    df = df.drop("left", axis=1)
    #3、特征选择
    #4、特征处理
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=["satisfaction_level","last_evaluation","number_project",\
                "average_monthly_hours","time_spend_company","Work_accident",\
                "promotion_last_5years"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
    scaler_lst=[slr,dp]
    column_lst=["salary","department"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=="salary":
                # low 为0,medium为1，high为2
                df[column_lst[i]]=[map_salary(s) for s in df["salary"].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df.values),label  
    return df,label            
                



features,label = hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,pl5=True,dp=True,slr=True)

## 建模函数

In [49]:
from sklearn.model_selection import train_test_split

def hr_modeling_nn(features,label):
    f_v = features.values
    f_names = features.columns.values
    l_v = label.values
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    print(len(X_train),len(X_train),print(X_validation))
    
    return X_train, X_test, Y_train, Y_test,X_validation, Y_validation
    
X_train, X_test, Y_train, Y_test,X_validation, Y_validation = hr_modeling_nn(features,label)

[[ 1.23547405 -1.3209701   0.97111292 ...  0.          0.
   1.        ]
 [ 0.47126283  1.42494396  0.97111292 ...  0.          1.
   0.        ]
 [-0.25272675 -0.67830936 -0.65153764 ...  0.          0.
   1.        ]
 ...
 [-0.73538647  0.25646989  1.7824382  ...  0.          0.
   0.        ]
 [ 0.02882475  1.60021507  0.15978764 ...  0.          0.
   0.        ]
 [ 1.35613898 -0.03564863  0.15978764 ...  1.          0.
   0.        ]]
8999 8999 None


## 机器学习 --分类

In [80]:
from sklearn.externals.six import StringIO
    
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
    
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.optimizers import SGD


f_v=features.values
f_names=features.columns.values
    
mdl = Sequential()
mdl.add(Dense(50,input_dim=len(f_v[0])))
mdl.add(Activation("sigmoid"))
mdl.add(Dense(2))
mdl.add(Activation("softmax"))
sgd=SGD(learning_rate=0.05)
mdl.compile(loss="mean_squared_error",optimizer="adam")
mdl.fit(X_train,np.array([[0,1] if i==1 else [1,0] for i in Y_train]),nb_epoch=2000,batch_size=2048)
xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
for i in range(len(xy_lst)):
    X_part=xy_lst[i][0]
    Y_part=xy_lst[i][1]
    Y_pred=mdl.predict_classes(X_part)
    print(i)
    print("NN","-ACC:",accuracy_score(Y_part,Y_pred))
    print("NN","-REC:",recall_score(Y_part,Y_pred))
    print("NN","-F1:",f1_score(Y_part,Y_pred))   

models = []
models.append(("KNN",KNeighborsClassifier(n_neighbors=3,n_jobs=-1)))
models.append(("GaussianNB",GaussianNB()))
models.append(("BernoulliNB",BernoulliNB()))
models.append(("DecisionTreeGini",DecisionTreeClassifier()))
models.append(("DecisionTreeEntropy",DecisionTreeClassifier(criterion="entropy")))
models.append(("SVM Classifier",SVC(C=1000)))
models.append(("OriginalRandomForest",RandomForestClassifier()))
models.append(("RandomForest",RandomForestClassifier(n_estimators=11,max_features=None,n_jobs=-1)))
models.append(("Adaboost",AdaBoostClassifier(n_estimators=100)))
# sag 随机梯度下降
models.append(("LogisticRegression",LogisticRegression(C=1000,tol=1e-10,solver="sag",max_iter=1000)))
models.append(("GBDT",GradientBoostingClassifier(max_depth=6,n_estimators=100)))
  
      
for clf_name,clf in models:
    clf.fit(X_train,Y_train)
    xy_lst=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
    for i in range(len(xy_lst)):
        X_part=xy_lst[i][0]
        Y_part=xy_lst[i][1]
        Y_pred=clf.predict(X_part)
        print(i)
        print(clf_name,"-ACC:",accuracy_score(Y_part,Y_pred))
        print(clf_name,"-REC:",recall_score(Y_part,Y_pred))
        print(clf_name,"-F1:",f1_score(Y_part,Y_pred)) 
#         if clf_name.startswith("DecisionTree"):
#             dot_data=StringIO()
#             export_graphviz(clf,out_file=dot_data,
#                                      feature_names=f_names,
#                                      class_names=["NL","L"],
#                                      filled=True,
#                                      rounded=True,
#                                      special_characters=True)
#             graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
#             graph.write_pdf("dt_tree_%s.pdf"%(clf_name))

/Users/hui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/2000
8999/8999 [==============================] - 0s 42us/step - loss: 0.1826
Epoch 2/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1784
Epoch 3/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1765
Epoch 4/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.1739
Epoch 5/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1715
Epoch 6/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1694
Epoch 7/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1672
Epoch 8/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1652
Epoch 9/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1633
Epoch 10/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1615
Epoch 11/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1599
Epoch 12/2000
8999/8999 [==============================] - 0s 

8999/8999 [==============================] - 0s 3us/step - loss: 0.1069
Epoch 97/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1058
Epoch 98/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1047
Epoch 99/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1036
Epoch 100/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.1025
Epoch 101/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.1013
Epoch 102/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.1002
Epoch 103/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0991
Epoch 104/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0979
Epoch 105/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0969
Epoch 106/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0957
Epoch 107/2000
8999/8999 [==============================] - 0

8999/8999 [==============================] - 0s 3us/step - loss: 0.0501
Epoch 191/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0500
Epoch 192/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0498
Epoch 193/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0496
Epoch 194/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0494
Epoch 195/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0492
Epoch 196/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0491
Epoch 197/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0489
Epoch 198/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0487
Epoch 199/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0486
Epoch 200/2000
8999/8999 [==============================] - 0s 3us/step - loss: 0.0484
Epoch 201/2000
8999/8999 [==============================] 

8999/8999 [==============================] - 0s 4us/step - loss: 0.0399
Epoch 285/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0399
Epoch 286/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0398
Epoch 287/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0397
Epoch 288/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0397
Epoch 289/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0396
Epoch 290/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0395
Epoch 291/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0395
Epoch 292/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0394
Epoch 293/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0394
Epoch 294/2000
8999/8999 [==============================] - 0s 4us/step - loss: 0.0393
Epoch 295/2000
8999/8999 [==============================] 

...

0
SVM Classifier -ACC: 0.9882209134348261
SVM Classifier -REC: 0.9674461256304447
SVM Classifier -F1: 0.9754969949144707
1
SVM Classifier -ACC: 0.9646666666666667
SVM Classifier -REC: 0.928374655647383
SVM Classifier -F1: 0.9270976616231086
2
SVM Classifier -ACC: 0.9676666666666667
SVM Classifier -REC: 0.9352409638554217
SVM Classifier -F1: 0.9275578790141896


/Users/hui/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0
OriginalRandomForest -ACC: 0.9986665185020558
OriginalRandomForest -REC: 0.994956441999083
OriginalRandomForest -F1: 0.9972426470588235
1
OriginalRandomForest -ACC: 0.9863333333333333
OriginalRandomForest -REC: 0.953168044077135
OriginalRandomForest -F1: 0.9712280701754387
2
OriginalRandomForest -ACC: 0.9863333333333333
OriginalRandomForest -REC: 0.9518072289156626
OriginalRandomForest -F1: 0.9685823754789271
0
RandomForest -ACC: 0.9985553950438938
RandomForest -REC: 0.9944979367262724
RandomForest -F1: 0.9970121811077914
1
RandomForest -ACC: 0.987
RandomForest -REC: 0.9586776859504132
RandomForest -F1: 0.9727463312368974
2
RandomForest -ACC: 0.9853333333333333
RandomForest -REC: 0.9548192771084337
RandomForest -F1: 0.9664634146341464
0
Adaboost -ACC: 0.962773641515724
Adaboost -REC: 0.9211370930765704
Adaboost -F1: 0.9230415805191822
1
Adaboost -ACC: 0.9543333333333334
Adaboost -REC: 0.8980716253443526
Adaboost -F1: 0.9049271339347675
2
Adaboost -ACC: 0.9583333333333334
Adaboost -RE

/Users/hui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0
LogisticRegression -ACC: 0.7879764418268697
LogisticRegression -REC: 0.35488308115543327
LogisticRegression -F1: 0.44791666666666663
1
LogisticRegression -ACC: 0.7776666666666666
LogisticRegression -REC: 0.32231404958677684
LogisticRegression -F1: 0.41233480176211457
2
LogisticRegression -ACC: 0.7906666666666666
LogisticRegression -REC: 0.3147590361445783
LogisticRegression -F1: 0.39961759082217974
0
GBDT -ACC: 0.9955550616735193
GBDT -REC: 0.9853278312700596
GBDT -F1: 0.9907791609036423
1
GBDT -ACC: 0.9846666666666667
GBDT -REC: 0.9504132231404959
GBDT -F1: 0.967741935483871
2
GBDT -ACC: 0.9846666666666667
GBDT -REC: 0.9548192771084337
GBDT -F1: 0.964992389649924


## 回归测试

In [70]:
# print("X",features)
# print("Y",label)
from sklearn.linear_model import LinearRegression,Ridge,Lasso
#regr=LinearRegression()
regr=Ridge(alpha=1)
regr.fit(features.values,label.values)
Y_pred=regr.predict(features.values)
print("Coef:",regr.coef_)
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
print("MSE:",mean_squared_error(label.values,Y_pred))
print("MAE:",mean_absolute_error(label.values,Y_pred))
print("R2:",r2_score(label.values,Y_pred))

Coef: [-0.16009483  0.01494029 -0.04196149  0.03202485  0.05320615 -0.05465984
 -0.01616189 -0.10639539  0.09246624  0.01392915 -0.01076429 -0.06050522
  0.0146708   0.04922629 -0.04760503  0.0122534  -0.00939767  0.00951584
  0.02061428  0.0219916 ]
MSE: 0.14230903029941022
MAE: 0.3019494008573236
R2: 0.2154927025916843
